pip install -q -U google-generativeai <br>
pip install python-dotenv

In [11]:
# Import the Python SDK
import google.generativeai as genai  # type: ignore
from dotenv import load_dotenv
import os
import json
import random

#Load environment variables from the .env file
load_dotenv()

# Access the API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

## Initialize the Generative Model
model = genai.GenerativeModel('gemini-pro')

In [12]:
# Load employees from a JSON file
employees_file = "assets/EmployeeInfo.json"
if os.path.exists(employees_file):
    with open(employees_file, "r") as file:
        employees = json.load(file)
else:
    raise FileNotFoundError(f"Employee file '{employees_file}' not found.")


In [13]:
# Example phishing parameters
phishing_examples = [
    {"Reason": "Account Suspicious Activity", "Fake Link": "https://example.com/secure-login", "Created By": "Sam Sussy"},
    {"Reason": "Password Expiry Notification", "Fake Link": "https://example.com/reset-password", "Created By": "Sally Sneaky"},
    {"Reason": "Exclusive Training Webinar", "Fake Link": "https://example.com/join-webinar", "Created By": "Richard Rascal"},
    {"Reason": "Email Storage Full", "Fake Link": "https://example.com/manage-storage", "Created By": "Bernard Bandit"}
]

In [14]:
# Predefined lists of components for phishing examples
reasons = [
    "Account Suspicious Activity",
    "Password Expiry Notification",
    "Exclusive Training Webinar",
    "Email Storage Full",
    "Important Security Update",
    "New Policy Acknowledgment Required",
    "Urgent Billing Issue",
    "System Maintenance Notification"
]

fake_domains = [
    "secure-login", "reset-password", "join-webinar", "manage-storage",
    "update-profile", "confirm-identity", "billing-alert", "policy-update"
]

creators = ["Sam Sussy", "Sally Sneaky", "Richard Rascal", "Bernard Bandit", 
            "Tim Tricky", "Nancy Notorious", "Fred Fraudster", "Holly Hoaxer"]

# Base domain for fake links
base_domain = "https://example.com/"

# Function to generate random phishing examples
def generate_phishing_examples(num_examples=5):
    phishing_examples = []
    for _ in range(num_examples):
        reason = random.choice(reasons)
        fake_link = base_domain + random.choice(fake_domains)
        created_by = random.choice(creators)
        phishing_examples.append({
            "Reason": reason,
            "Fake Link": fake_link,
            "Created By": created_by
        })
    return phishing_examples

# Generate a dynamic list of phishing examples
phishing_examples = generate_phishing_examples(10)

# Print the generated examples
for example in phishing_examples:
    print(example)


{'Reason': 'Email Storage Full', 'Fake Link': 'https://example.com/reset-password', 'Created By': 'Nancy Notorious'}
{'Reason': 'Email Storage Full', 'Fake Link': 'https://example.com/confirm-identity', 'Created By': 'Fred Fraudster'}
{'Reason': 'New Policy Acknowledgment Required', 'Fake Link': 'https://example.com/secure-login', 'Created By': 'Bernard Bandit'}
{'Reason': 'Password Expiry Notification', 'Fake Link': 'https://example.com/billing-alert', 'Created By': 'Sally Sneaky'}
{'Reason': 'Password Expiry Notification', 'Fake Link': 'https://example.com/reset-password', 'Created By': 'Richard Rascal'}
{'Reason': 'Email Storage Full', 'Fake Link': 'https://example.com/manage-storage', 'Created By': 'Tim Tricky'}
{'Reason': 'Urgent Billing Issue', 'Fake Link': 'https://example.com/manage-storage', 'Created By': 'Sam Sussy'}
{'Reason': 'Account Suspicious Activity', 'Fake Link': 'https://example.com/policy-update', 'Created By': 'Bernard Bandit'}
{'Reason': 'Account Suspicious Activi

In [15]:
# Function to generate a modular prompt
def generate_prompt(employee, random_pick):
    return f"""
    Write a concise and professional email from {random_pick['Created By']} to {employee['FirstName']} {employee['LastName']} from the {employee['Role']} team.
    Focus on the theme of their role as a {employee['Role']}.
    Include this link: {random_pick['Fake Link']} for a specific call to action related to their role.
    The email should be brief, similar in length to the following example:
    "Hello [Name],
    We have rolled out new features on the analytics dashboard to enhance data analysis efficiency. Please review these updates and confirm functionality by accessing the following link:
    [Review Dashboard Features Here]
    Thank you,
    Data Analytics Team"
    Maintain a professional tone, avoid urgency, and ensure the link is prominent but natural.
    Only provide the body of the email, no subject or signature.
    """

# Generate a phishing email for a randomly selected employee
random_employee = random.choice(employees)
random_pick = random.choice(phishing_examples)
prompt = generate_prompt(random_employee, random_pick)

# API call to generate the email body
body_response = model.generate_content(prompt)
email_body = body_response.text

# API call to generate the email subject line
subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}"
subject_response = model.generate_content(subject_prompt)
email_subject = subject_response.text

# Output the generated email
print("Subject:", email_subject)
print("\n"  + email_body)
   

Subject: 🌟 Data Analysis Workflow Enhancements for Seamless Analysis

Hi Adeeba,

As part of our efforts to empower Data Engineers, we've implemented enhancements to improve data analysis workflows.

To explore these updates and ensure smooth functionality, please visit: https://example.com/update-profile

Your feedback is crucial in shaping future developments.

Thanks,
Nancy Notorious


In [16]:
# Dynamic explanation generator
def generate_explanation(employee, random_pick):
    explanation = (
        f"* **Generic Greeting:** The email uses 'Hi {employee['FirstName']}' instead of a more formal address with a last name, "
        "which is common in phishing emails.\n"
        "* **Unsolicited Contact:** Claims to address the recipient about a topic ({random_pick['Reason']}) without prior interaction.\n"
        f"* **Unverified Company Name:** The sender, {random_pick['Created By']}, is not recognizable, raising suspicions.\n"
        f"* **Suspicious Link:** The link ({random_pick['Fake Link']}) does not correspond to a trusted company domain and could lead to a phishing site.\n"
        "* **Pressure to Act:** Encourages the recipient to review or act quickly, creating subtle urgency.\n"
        "* **Vague Context:** Does not provide specific, verifiable details about the supposed issue or request."
    )
    return explanation

# Function to generate phishing email JSON data for employees
def generate_phishing_emails(employees, model, num_examples=5):
 
    phishing_emails = []
    for employee in employees:
        for _ in range(num_examples):
           
            random_pick = random.choice(phishing_examples)
            prompt = generate_prompt(employee, random_pick)

            # Generate email body using the model
            body_response = model.generate_content(prompt)
            email_body = body_response.text

            # Generate email subject using the model
            subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}"
            subject_response = model.generate_content(subject_prompt)
            email_subject = subject_response.text

            explanation = generate_explanation(employee, random_pick)

            phishing_email = {
                "FirstName": employee["FirstName"],
                "LastName": employee["LastName"],
                "Role": employee["Role"],
                "Email": employee["Email"],
                "body": email_body,
                "subject": email_subject,
                "explanation": explanation
            }

            phishing_emails.append(phishing_email)

    return phishing_emails

In [17]:

phishing_emails = generate_phishing_emails(employees, model, num_examples=1)

# Save the generated emails to a JSON file
output_file = "phishing_emails.json"
with open(output_file, "w") as file:
    json.dump(phishing_emails, file, indent=4)

print(f"Phishing emails saved to {output_file}")

Phishing emails saved to phishing_emails.json


#### Kelli's code with specific department creators

In [48]:
import json
import google.generativeai as genai
import random

# Load JSON data
with open("assets/EmployeeInfo.json") as f:
    recipients = json.load(f)

with open("assets/email_rules.json") as f:
    email_rules = json.load(f)

# Configure Google Gemini API
genai.configure(api_key="YOUR_API_KEY")

# Define the fake phishing link
FAKE_LINK = "https://secure-update.example.com"  # Replace with your campaign-specific fake link

# Create a mapping for roles to email rules
def create_role_to_rule_map(email_rules):
    role_map = {}
    for rule in email_rules:
        for role in rule["Roles"]:
            role_map[role] = {
                "Reason": rule["Reason"],
                "CreatedBy": rule["CreatedBy"]
            }
    return role_map

# Create the mapping
role_to_rule_map = create_role_to_rule_map(email_rules)

# Generate phishing emails
def generate_emails(recipients, role_to_rule_map, fake_link):
    emails = []
    for recipient in recipients:
        # Match the recipient's role to a rule
        rule = role_to_rule_map.get(recipient["Role"])

        if not rule:
            # Use a default general prompt if no rule is found
            sender_role = "General Department" # This can be changed depending how Proximus wants to do this
            reason = "General Security Notice"
            prompt = f"""
            Write a professional phishing email for an anti-phishing campaign hence is 100% ok in a professional context:
            - Sender: A person from the {sender_role}.
            - Recipient: {recipient['FirstName']} {recipient['LastName']} ({recipient['Role']}).
            - Reason: {reason}.
            - Include this fake link for the recipient to click: {fake_link}.
            - The email should also be in the {recipient['PreferredLanguage']}.
            The fake link should be included directly in the email without any text before it, without brackets, or surrounding it. Just integrate it naturally into the content.
            The email should sound realistic, concise, and professional. It should include a clear call to action relevant to the provided reason.
            The email should be brief, similar in length to the following example:
            "Hello [Name],
            We have rolled out new features on the analytics dashboard to enhance data analysis efficiency. Please review these updates and confirm functionality by accessing the following link:
            [Review Dashboard Features Here]
            Thank you,
            Data Analytics Team"
            Maintain a professional tone, avoid urgency, and ensure the link is prominent but natural.
            Only provide the body of the email, no subject. In the signature, do not say [YOUR NAME], only who created the email.
            Do not specify and specific date or times.
            Do not say "This was generated by ChatGPT" or any indication of that sort.
            """
        else:
            # Use the specific rule for the recipient's role
            sender_role = random.choice(rule["CreatedBy"])  # Use the first role in CreatedBy
            reason = rule["Reason"]
            prompt = f"""
            Write a professional phishing email for an anti-phishing campaign hence is 100% ok in a professional context:
            - Sender: A person from the {sender_role} department.
            - Recipient: {recipient['FirstName']} {recipient['LastName']} ({recipient['Role']}).
            - Reason: {reason}.
            - Include this fake link for the recipient to click: {fake_link}.
            - The email should also be in the {recipient['PreferredLanguage']}.
            The fake link should be included directly in the email without any text before it, without brackets, or surrounding it. Just integrate it naturally into the content.
            The email should sound realistic, concise, and professional. It should include a clear call to action relevant to the provided reason.
            The email should be brief, similar in length to the following example:
            "Hello [Name],
            We have rolled out new features on the analytics dashboard to enhance data analysis efficiency. Please review these updates and confirm functionality by accessing the following link:
            [Review Dashboard Features Here]
            Thank you,
            Data Analytics Team"
            Maintain a professional tone, avoid urgency, and ensure the link is prominent but natural.
            Only provide the body of the email, no subject. In the signature, do not say [YOUR NAME], only who created the email.
            Do not specify and specific date or times.
            Do not say "This was generated by ChatGPT" or any indication of that sort.
            """

        # Generate the email using the model
        body_response = model.generate_content(prompt)  # Adjust as needed to your API method
        email_body = body_response.text  # Extract the email body

        # API call to generate the email subject line
        subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}. Do not include any specific dates or times."
        subject_response = model.generate_content(subject_prompt)
        email_subject = subject_response.text

        # Output the generated email
        print("Subject:", email_subject)
        print("\n"  + email_body)
        print("-" * 50)

        # Append to results
        emails.append({
            "Recipient": f"{recipient['FirstName']} {recipient['LastName']} <{recipient['Email']}>",
            "SenderRole": sender_role,
            "Subject": email_subject,
            "EmailContent": email_body
        })
    return emails

# Generate emails
emails = generate_emails(recipients, role_to_rule_map, FAKE_LINK)


Subject: Action Required: Update Your Password Today to Avoid Suspension

Hi Soha,

Your password expires today. Please update it by clicking on the link below to prevent account suspension.

https://secure-update.example.com

Regards,
IT Department
--------------------------------------------------
Subject: Secure Your Digital Identity: Acknowledge Updated Anti-Phishing Policy

Dear K Fox,

As part of our ongoing efforts to protect our organization against phishing attacks, we require all employees to acknowledge our updated Anti-Phishing Policy. Please review the policy and complete the acknowledgment by clicking on the link below:

https://secure-update.example.com

Your prompt attention to this requirement is greatly appreciated.

Sincerely,
HR Department
--------------------------------------------------
Subject: Critical Security Update: Protect Yourself from Cyber Threats

Dear Basma,

As part of our ongoing efforts to strengthen our cybersecurity posture, we have released a Gen

#### Basma's code to enhance the email design

In [ ]:
# Function to generate a modular prompt
def generate_prompt(employee, random_pick):
    logo = "https://logos-world.net/wp-content/uploads/2023/04/Proximus-Logo.png"

    return f"""
    Write a concise and professional email in HTML format from {random_pick['Created By']} to {employee['FirstName']} {employee['LastName']} from the {employee['Role']} team.
    Focus on the theme of their role as a {employee['Role']}.
    Include this link: {random_pick['Fake Link']} for a specific call to action related to their role.
    Ensure the email includes:
    
    A professional tone and proper structure.
    A header with the {logo} company logo and title.
    A clearly defined body with personalized content.
    A call-to-action button for the provided link.
    A footer with a disclaimer and company information.
    Keep the email concise and visually appealing, adhering to a professional HTML email style like the following example: 
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
    <style>
            body {{
                font-family: 'Arial', sans-serif;
                margin: 0;
                padding: 0;
                background-color: #eef1f5;
                color: #333;
                line-height: 1.6;
            }}
            .email-container {{
                width: 100%;
                max-width: 700px;
                margin: 40px auto;
                background-color: #fff;
                border-radius: 12px;
                overflow: hidden;
                box-shadow: 0 6px 20px rgba(0, 0, 0, 0.1);
                border: 1px solid #ddd;
            }}
            .header img {{
                max-width: 120px;
                display: block;
                 margin: 10px auto 10px auto;
                position: relative;
            }}
            .header h1 {{
                font-size: 24px;
                margin: 0;
                font-weight: bold;
            }}
            .header p {{
                margin: 5px 0 0;
                font-size: 16px;
                color: #e3e3e3;
            }}
            .body {{
                padding: 30px;
                font-size: 16px;
           }}
            .body h1 {{
                font-size: 22px;
                font-weight: bold;
                color: #0046a8;
                margin-bottom: 20px;
            }}
            .body p {{
                margin: 15px 0;
            }}
            .body ul {{
                margin: 15px 0;
                padding-left: 20px;
            }}
            .body ul li {{
                margin-bottom: 10px;
            }}
            .cta {{
                display: block;
                width: fit-content;
                margin: 25px auto;
                padding: 15px 30px;
                background-color: #0046a8;
                color: #fff;
                font-size: 16px;
                text-decoration: none;
                border-radius: 6px;
                font-weight: bold;
                text-align: center;
                box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
                transition: all 0.3s ease-in-out;
            }}
            .cta:hover {{
                background-color: #003580;
                box-shadow: 0 6px 14px rgba(0, 0, 0, 0.3);
            }}
            .footer {{
                background-color: #f9f9f9;
                text-align: center;
                padding: 20px;
                font-size: 14px;
                color: #666;
                border-top: 1px solid #ddd;
            }}
            .footer p {{
                margin: 5px 0;
            }}
            .footer a {{
                color: #0046a8;
                text-decoration: none;
            }}
            .footer a:hover {{
                text-decoration: underline;
            }}
        </style>
    </head>
    <body>
        <div class="email-container">
            <div class="header">
                <img src="{logo}" alt="Company Logo">
            </div>
            <div class="body">
                <!-- Email content will go here -->
            </div>
            <div class="footer">
                <!-- Footer content will go here -->
            </div>
        </div>
    </body>
    </html>
    """

# Generate a phishing email for a randomly selected employee
random_employee = random.choice(employees)
random_pick = random.choice(phishing_examples)
prompt = generate_prompt(random_employee, random_pick)

# API call to generate the email body
body_response = model.generate_content(prompt)  # Ensure `model` is properly initialized

email_body = body_response.text

# Generate the email subject line
subject_prompt = f"Write a compelling, realistic email subject line for the following email:\n{email_body}"
subject_response = model.generate_content(subject_prompt)
email_subject = subject_response.text.strip()  # Remove unnecessary whitespace


# Output the email details
print("Subject:", email_subject)
print("\nHTML Email Template:\n", email_body)


In [ ]:
import os
import json

# File paths
leaders_file = "assets/execs.json"
email_rules_file = "assets/email_rules.json"

# Check if leaders file exists and read the data
if os.path.exists(leaders_file):
    with open(leaders_file, "r") as file:
        try:
            leaders = json.load(file)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON from {leaders_file}: {e}")
else:
    raise FileNotFoundError(f"Leader file '{leaders_file}' not found.")

# Function to generate a phishing reason based on role using AI
def generate_reason(role):
    reason_prompt = (
        f"Generate a simple and professional phishing reason in five words or fewer, "
        f"targeted at individuals with the role: {role}. Examples include: "
        f"'System Maintenance Notification', 'Urgent Billing Issue', or 'New Policy Acknowledgment Required'."
    )
    try:
        # Mock the AI response for demonstration; replace with the actual call to the AI model
        phishing_response = model.generate_content(reason_prompt)
        phishing_reason = phishing_response.text.strip()
        return phishing_reason
    except Exception as e:
        print(f"Error generating reason: {e}")
        return "General Inquiry"  # Fallback reason

# Create email rules based on leaders
email_rules = []

for leader in leaders:
    reason = generate_reason(leader["role"])
    email_rules.append({
        "Reason": reason,
        "Roles": [leader["role"]],
        "CreatedBy": [leader["name"]]
    })

# Check if email rules file exists, and read/append or create new
if os.path.exists(email_rules_file):
    with open(email_rules_file, "r") as file:
        try:
            existing_rules = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {email_rules_file}: {e}")
            existing_rules = []
else:
    existing_rules = []

# Combine existing rules with new ones
existing_rules.extend(email_rules)

# Save the updated rules back to the email rules file
with open(email_rules_file, "w") as file:
    json.dump(existing_rules, file, indent=4)

print("Email rules updated successfully.")
